<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/11th%20April%202nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
import requests
from scipy import stats

import lightgbm as lgb

In [7]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [8]:
feat= [f for f in train.columns if "feature" in f]

In [9]:
train_all = pd.concat([train,target], axis=1 )

In [10]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

In [11]:
!pip install optuna

In [13]:
train_all= pd.concat([train, target], axis=1)
test_data = train_all[train_all['moons'] > 37]

In [14]:
targets= ['target_r', 'target_g', 'target_b']

In [15]:
test_data

,id,moons,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,target_r,target_g,target_b
86240,0x76ad39448d364e4d,38,0.00,0.25,0.00,0.25,0.50,0.50,0.00,0.00,0.25,0.25,0.25,0.25,0.50,0.00,0.00,0.00,0.25,0.25,0.25,0.25,0.00,0.50,0.25,0.00,0.50,0.00,0.75,0.00,0.00,0.75,1.00,1.00
86241,0xbd3b9b5ff1d29050,38,0.25,1.00,1.00,1.00,1.00,1.00,0.00,0.50,1.00,1.00,1.00,1.00,1.00,0.25,1.00,1.00,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.25,1.00,0.50,0.25,0.75,0.75,0.25,0.00,0.25
86242,0x49a71beb6a10e346,38,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.75,0.75,1.00,0.50,0.50,0.75,0.75,1.00
86243,0x8e29753f227bbd18,38,0.00,0.50,0.50,0.50,0.75,0.75,0.00,0.00,0.50,0.75,0.50,0.75,0.75,0.25,0.50,0.00,0.25,0.25,0.25,0.25,0.25,0.50,0.50,0.00,0.50,0.00,0.25,0.25,0.00,0.75,0.75,1.00
86244,0x6b163256a81cb13d,38,0.25,1.00,1.00,1.00,1.00,1.00,0.25,0.25,1.00,1.00,1.00,1.00,1.00,0.00,0.25,0.75,0.50,0.50,0.50,0.25,1.00,0.75,0.75,0.00,0.50,1.00,1.00,0.00,0.25,0.25,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91070,0xaf153a0e67d184fd,39,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.50,0.25,0.50,0.50,0.25,0.50,0.75,0.75
91071,0x2c71a916f9382281,39,0.50,0.25,0.25,0.25,0.25,0.25,0.75,0.75,0.00,0.25,0.00,0.25,0.25,0.00,0.25,0.50,0.75,0.75,0.75,0.00,0.50,0.25,0.25,0.75,0.50,0.75,0.50,0.25,0.75,0.25,0.50,0.00
91072,0xb78084cc4181a581,39,1.00,0.25,0.50,0.25,0.00,0.00,1.00,0.75,0.00,0.50,0.25,0.00,0.00,0.00,0.50,1.00,0.50,0.50,0.50,0.25,0.50,0.00,0.00,1.00,1.00,0.50,0.50,0.50,0.25,1.00,1.00,0.25
91073,0x525077079cf03613,39,0.75,0.25,0.25,0.25,0.50,0.50,0.75,0.25,0.00,0.25,0.00,0.25,0.25,0.00,0.25,0.25,0.25,0.25,0.25,0.00,0.25,0.00,0.25,1.00,0.50,1.00,0.75,0.00,0.25,0.00,1.00,0.75


In [16]:
import optuna

train = train_all[train_all['moons'] <= 37]
X_train, X_test, y_train, y_test = train_test_split(train[feat], train[targets], test_size=0.25, shuffle=False)

tr_data_r = lgb.Dataset(X_train, y_train.target_r)
va_data_r = lgb.Dataset(X_test, y_test.target_r)

tr_data_g = lgb.Dataset(X_train, y_train.target_g)
va_data_g = lgb.Dataset(X_test, y_test.target_g)

tr_data_b = lgb.Dataset(X_train, y_train.target_b)
va_data_b = lgb.Dataset(X_test, y_test.target_b)


N_TRIALS = 200
def objective(trial):    
    
    params = {
    'num_leaves': trial.suggest_int('num_leaves', 4, 64),
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': trial.suggest_int('max_depth', 4, 16),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 16),
    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
    'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
    'min_child_samples': trial.suggest_int('min_child_samples', 4, 80),
    'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
    'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
    'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
    'max_bin': trial.suggest_int('max_bin', 50, 900),
    'early_stopping_rounds': 100, 
    }


    model_r = lgb.train(params, tr_data_r, valid_sets=[tr_data_r, va_data_r], verbose_eval=None, num_boost_round=10000)
    model_g = lgb.train(params, tr_data_g, valid_sets=[tr_data_g, va_data_g], verbose_eval=None, num_boost_round=10000)
    model_b = lgb.train(params, tr_data_b, valid_sets=[tr_data_b, va_data_b], verbose_eval=None, num_boost_round=10000)

    # score_r = scorer(y_test.target_r, model_r.predict(X_test[feat]))
    # score_g = scorer(y_test.target_g, model_g.predict(X_test[feat]))
    # score_b = scorer(y_test.target_b, model_b.predict(X_test[feat]))
    # print((score_r+score_g+score_b)/ 3)

    score_r = scorer(test_data.target_r, model_r.predict(test_data[feat]))
    score_g = scorer(test_data.target_g, model_g.predict(test_data[feat]))
    score_b = scorer(test_data.target_b, model_b.predict(test_data[feat]))

    # print((score_r+score_g+score_b)/ 3)
    score = (score_r+score_g+score_b)/ 3

    print(score)
    
    from google.colab import output
    output.clear()
    return score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar= True)
display(study.best_params)
display(study.best_trial.value)

[I 2021-04-11 19:17:04,347] Trial 199 finished with value: 0.4325733494295292 and parameters: {'num_leaves': 59, 'max_depth': 5, 'min_child_weight': 2, 'feature_fraction': 0.44539757530781915, 'bagging_fraction': 0.48101050663031153, 'bagging_freq': 6, 'min_child_samples': 54, 'lambda_l1': 3.940334935469895e-07, 'lambda_l2': 1.3267907379825841e-08, 'learning_rate': 0.09389197642540993, 'max_bin': 468}. Best is trial 84 with value: 0.43994241981193055.



{'bagging_fraction': 0.8076283910483392,
 'bagging_freq': 5,
 'feature_fraction': 0.43447838030929503,
 'lambda_l1': 1.3042421656139844e-06,
 'lambda_l2': 2.721530697637648e-06,
 'learning_rate': 0.00014457017130537278,
 'max_bin': 799,
 'max_depth': 14,
 'min_child_samples': 66,
 'min_child_weight': 7,
 'num_leaves': 59}

0.43994241981193055

In [17]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'bagging_fraction': 0.8076283910483392,
    'bagging_freq': 5,
    'feature_fraction': 0.43447838030929503,
    'lambda_l1': 1.3042421656139844e-06,
    'lambda_l2': 2.721530697637648e-06,
    'learning_rate': 0.00014457017130537278,
    'max_bin': 799,
    'max_depth': 14,
    'min_child_samples': 66,
    'min_child_weight': 7,
    'num_leaves': 59
    }

In [18]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [19]:
feat= [f for f in train.columns if "feature" in f]

In [20]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train[feat], target[targets], test_size=0.25, shuffle=False)
tr_data_r = lgb.Dataset(X_train, y_train.target_r)
va_data_r = lgb.Dataset(X_test, y_test.target_r)

tr_data_g = lgb.Dataset(X_train, y_train.target_g)
va_data_g = lgb.Dataset(X_test, y_test.target_g)

tr_data_b = lgb.Dataset(X_train, y_train.target_b)
va_data_b = lgb.Dataset(X_test, y_test.target_b)

model_r = lgb.train(params, tr_data_r, valid_sets=[tr_data_r, va_data_r], verbose_eval=2000, num_boost_round=10000, early_stopping_rounds=100)
model_g = lgb.train(params, tr_data_g, valid_sets=[tr_data_g, va_data_g], verbose_eval=2000, num_boost_round=10000, early_stopping_rounds=100)
model_b = lgb.train(params, tr_data_b, valid_sets=[tr_data_b, va_data_b], verbose_eval=2000, num_boost_round=10000, early_stopping_rounds=100)


Training until validation scores don't improve for 100 rounds.
[2000]	training's rmse: 0.340662	valid_1's rmse: 0.350312
[4000]	training's rmse: 0.333768	valid_1's rmse: 0.344194
[6000]	training's rmse: 0.329184	valid_1's rmse: 0.340471
[8000]	training's rmse: 0.326031	valid_1's rmse: 0.338215
[10000]	training's rmse: 0.323792	valid_1's rmse: 0.336872
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.323792	valid_1's rmse: 0.336872
Training until validation scores don't improve for 100 rounds.
[2000]	training's rmse: 0.335042	valid_1's rmse: 0.332316
[4000]	training's rmse: 0.322834	valid_1's rmse: 0.319529
[6000]	training's rmse: 0.314877	valid_1's rmse: 0.311573
[8000]	training's rmse: 0.309565	valid_1's rmse: 0.30665
[10000]	training's rmse: 0.305935	valid_1's rmse: 0.303651
Did not meet early stopping. Best iteration is:
[10000]	training's rmse: 0.305935	valid_1's rmse: 0.303651
Training until validation scores don't improve for 100 rounds.
[2000]	training'

In [24]:
prediction= pd.DataFrame()
prediction['target_r'] = model_r.predict(test[feat])
prediction['target_g'] = model_g.predict(test[feat])
prediction['target_b'] = model_b.predict(test[feat])

In [25]:
prediction[prediction['target_r'] > 1] =1
prediction[prediction['target_g'] > 1] =1
prediction[prediction['target_b'] > 1] =1

In [26]:
prediction[prediction['target_r'] < 0] = 0
prediction[prediction['target_g'] < 0] = 0
prediction[prediction['target_b'] < 0] = 0


In [27]:
display(prediction[prediction['target_r'] > 1],prediction[prediction['target_g'] > 1], prediction[prediction['target_b'] > 1])

,target_r,target_g,target_b


,target_r,target_g,target_b


,target_r,target_g,target_b


In [28]:
display(prediction[prediction['target_r'] < 0],prediction[prediction['target_g'] < 0], prediction[prediction['target_b'] < 0])

,target_r,target_g,target_b


,target_r,target_g,target_b


,target_r,target_g,target_b


In [29]:
API_KEY = "0GIskIC9Rd1I8sMw6nStutbIbMx7AaZzAyNfuMOVAVWgJtNa1IVQHZAD3WPj" # <- HERE

r = requests.post("https://tournament.datacrunch.com/api/submission",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same as the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted :)
